__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/pandasklar/blob/master/jupyter/13_Analyse_Datatypes.ipynb)__

# Analyse Datatypes
* `analyse_datatypes`: Returns info about the datatypes and the mem_usage of the columns of a DataFrame 
* `analyse_values`: Returns statistical data for a DataFrame, a Series or an Index 
* `analyse_cols`: Describes the datatypes and the content of a DataFrame. Merged info from analyse_datatypes and analyse_values
* `change_datatype`: Converts the datatypes of a DataFrame or a Series. Automatically, if you want.
* `copy_datatype`: Copies the dtypes from one dataframe to another, matching the column names.

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 21:39:58


In [2]:
import numpy      as np
import pandas     as pd 
import bpyth      as bpy

# pandasklar
try:
    import pandasklar as pak 
except ImportError as e:
    !pip install pandasklar
    import pandasklar as pak   
    
# verbose
pak.Config.set('VERBOSE', True)

# copy_on_write
pd.set_option("mode.copy_on_write", True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions



In [3]:
# Generate random data
anz = 10000
a = pak.random_series( anz, 'int',   min=-500, max=100 )
b = pak.random_series( anz, 'int',   min=-127, max=127, p_dup=0 ) # keine Dups erlaubt
c = a + b
a = a % 10 * 10
v = pak.random_series( anz, 'name',                  p_nan=0)
w = v.str[:1]
s = pak.random_series( anz, 'string',                p_nan=0)
t = pak.random_series( anz, 'string',                p_nan=0.1)
m = pak.random_series( anz, 'int',   min=0, max=127, p_nan=0.1 )
n = pak.random_series( anz, 'float',                 p_nan=0.2 ) #* 70000
o = pak.random_series( anz, 'choice', choice=['Bremen','Bremerhaven'], p_nan=0.3,   p_dup=0     )
p = pak.random_series( anz, 'list',                                    p_nan=0.1,   p_dup=0.5   )
z = pak.random_series( anz, 'mix',                                     p_nan=0.1,   p_dup=0     )

df = pak.dataframe( [a, b, c, v, w, s, t, m, n, o, p, z], verbose=False)
df.columns = ['int_grob','int_fein','float_summe','first_name','Letter1','string_nonan','string_nan','int_nan','float_nan','City','List','Mix']
df.float_summe = df.float_summe.astype('float')
pak.sample(df)

,int_grob,int_fein,float_summe,first_name,Letter1,string_nonan,string_nan,int_nan,float_nan,City,List,Mix
0,60,-103,-467.0,Louis,L,UtREÜiC,24zBB,113,NaN,NaN,"[Caroline, Britta, Werner, Tom, Arne, Sven, Mo...",{0}
1,20,75,-123.0,Tom,T,pÖWGPHf,Osva,38,0.600,Bremerhaven,"[Thorsten, Gertrud, Tanja, Pauline, Leon, Yvon...",-10908
3,50,12,-53.0,Florian,F,1oT9,ypajXp,105,NaN,Bremerhaven,"[Miriam, Tom, Maya, Anna, Irmgard, Kerstin]",NaN
5,10,-62,-561.0,Jennifer,J,häWau5,Qhdg,0,0.738,Bremen,"[Horst, Anja, Katharina]",Bremen
10,90,-16,-507.0,Maria,M,8orrOhW,<NA>,82,0.550,NaN,"[Leah, Rebecca, Charlotte, Angelika, Heinz, Fr...",hJut6xpfYM14ZMk1iCrp
19,0,73,-87.0,Thorsten,T,Xlügh6,iOiYIÖi,123,0.486,NaN,"[Charlotte, Walter, Carina, Elena, Tom, Tom, J...",-17248
43,50,21,-14.0,Zoe,Z,QecAAiI,75UTFG,<NA>,0.160,Bremerhaven,"[Ingeborg, Tom, Daniela, Toni, Tom, Anna, Brit...",äS8jÄNT2hz
47,40,71,-295.0,Marco,M,qi6hkv,7Uw6,127,0.721,Bremen,"[Bettina, Henri, Ali, Annette, Anja, Björn, Ka...",Bremerhaven
58,30,-41,-268.0,Curt,C,tR43P,IoomaO,55,0.133,NaN,"[Louise, Tom, Markus]",yqLWöBq
63,20,66,-362.0,Mathias,M,2VJF,Ü9Äve,66,0.582,NaN,"[Tom, Hertha, Nicole, Anja, Arne, Tanja]","[Robert, Hans, Louise, Ina, Christina, Ida, Ay..."


## analyse_datatypes()

In [4]:
?pak.analyse_datatypes

Signature: pak.analyse_datatypes(df, with_index=True)
Docstring: Returns info about the datatypes and the mem_usage of the columns of a DataFrame.  
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function


In [5]:
a = pak.analyse_datatypes(df)
a

,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_hashable,nan_allowed,mem_usage
col_no,,,,,,,,,
0,__index__,int64,np.int64,int64,True,False,True,False,80.0 B
1,int_grob,int64,np.int64,int64,True,False,True,False,80.0 B
2,int_fein,int64,np.int64,int64,True,False,True,False,80.0 B
3,float_summe,float64,np.float64,float64,True,False,True,True,80.0 B
4,first_name,str,pd.string,string,False,True,True,True,624.0 B
5,Letter1,str,pd.string,string,False,True,True,True,580.0 B
6,string_nonan,str,pd.string,string,False,True,True,True,767.0 B
7,string_nan,str,pd.string,string,False,True,True,True,780.0 B
8,int_nan,int64,pd.Int64,Int64,True,False,True,True,90.0 B


## analyse_values()

In [6]:
?pak.analyse_values

Signature:
pak.analyse_values(
    data,
    as_list=False,
    as_dict=False,
    sort=False,
    with_index=True,
    nanless_ints=False,
)
Docstring: Returns statistical data for a DataFrame, a Series or an Index     
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function


In [7]:
pak.analyse_values(df)

,col_name,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum,datatype_suggest
col_no,,,,,,,,,,,,
0,__index__,1,10000,0,0,10000,0.0,4999.50,4999.50,9999.0,49995000.00,np.int16
1,int_grob,1,10,0,9990,10000,0.0,44.62,40.00,90.0,446150.00,pd.Int8
2,int_fein,1,255,0,9745,10000,-127.0,-0.21,0.00,127.0,-2140.00,pd.Int8
3,float_summe,1,823,0,9177,10000,-627.0,-201.43,-203.00,224.0,-2014255.00,pd.Int16
4,first_name,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN,
5,Letter1,1,24,0,9976,10000,A,NaN,NaN,Z,NaN,
6,string_nonan,1,10000,0,0,10000,002z,NaN,NaN,üüÖpAI1,NaN,
7,string_nan,1,8987,1013,0,10000,01ÄqpuZ,NaN,NaN,üüVmdp,NaN,
8,int_nan,1,128,990,8882,10000,0.0,62.95,63.00,127.0,567156.00,pd.Int8


## analyse_cols()

In [8]:
?pak.analyse_cols

Signature: pak.analyse_cols(df, sort=False, with_index=True)
Docstring:
Describes the datatypes and the content of a DataFrame.
Merged info from analyse_datatypes and analyse_values.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function


In [9]:
pak.analyse_cols(df)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int16,True,False,True,False,80.0 B,1,10000,0,0,10000,0.0,4999.50,4999.50,9999.0,49995000.00
1,int_grob,int64,np.int64,int64,pd.Int8,True,False,True,False,80.0 B,1,10,0,9990,10000,0.0,44.62,40.00,90.0,446150.00
2,int_fein,int64,np.int64,int64,pd.Int8,True,False,True,False,80.0 B,1,255,0,9745,10000,-127.0,-0.21,0.00,127.0,-2140.00
3,float_summe,float64,np.float64,float64,pd.Int16,True,False,True,True,80.0 B,1,823,0,9177,10000,-627.0,-201.43,-203.00,224.0,-2014255.00
4,first_name,str,pd.string,string,,False,True,True,True,618.0 B,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN
5,Letter1,str,pd.string,string,,False,True,True,True,580.0 B,1,24,0,9976,10000,A,NaN,NaN,Z,NaN
6,string_nonan,str,pd.string,string,,False,True,True,True,728.0 B,1,10000,0,0,10000,002z,NaN,NaN,üüÖpAI1,NaN
7,string_nan,str,pd.string,string,,False,True,True,True,652.0 B,1,8987,1013,0,10000,01ÄqpuZ,NaN,NaN,üüVmdp,NaN
8,int_nan,int64,pd.Int64,Int64,pd.Int8,True,False,True,True,90.0 B,1,128,990,8882,10000,0.0,62.95,63.00,127.0,567156.00
9,float_nan,float64,np.float64,float64,np.float32,True,False,True,True,80.0 B,1,1000,2020,6980,10000,0.0,0.50,0.51,1.0,4011.86


## change_datatype()

In [10]:
?pak.change_datatype

Signature:
pak.change_datatype(
    data,
    search=None,
    verbose=None,
    msg='',
    category_maxsize=-1,
    nanless_ints=False,
)
Docstring:
Converts the datatypes of a DataFrame or a Series.
If used with a Series:    
Similar behavior as pandas astype. But it also accepts
sloppy class names like type_info knows.
If no target datatype is specified, it will be selected automatically.
If used with a DataFrame:
Converts all datatypes automatically.                      

* category_maxsize: How big can a category get to be suggested as datatype_suggest?
* nanless_ints: Are numpy's integer classes (that don't know NaN) suggested as datatype_suggest?    
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/analyse.py
Type:      function


In [11]:
# example Series, before
pak.analyse_datatypes(df.int_fein)

,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_hashable,nan_allowed,mem_usage
0,int_fein,int64,np.int64,int64,True,False,True,False,78.1 KB


In [12]:
# example Series, after 
a = pak.change_datatype(df.int_fein)
pak.analyse_datatypes(a)

int_fein             --> pd.Int8   


,col_name,datatype_instance,datatype,datatype_short,is_numeric,is_string,is_hashable,nan_allowed,mem_usage
0,int_fein,int8,pd.Int8,Int8,True,False,True,True,19.5 KB


In [13]:
# whole DataFrame
df2 = pak.change_datatype(df)
pak.analyse_cols(df2)

change_datatype 
int_grob             --> pd.Int8   
int_fein             --> pd.Int8   
float_summe          --> pd.Int16  
int_nan              --> pd.Int8   
float_nan            --> np.float32
City                 --> pd.string 
change_datatype before: 4.9 MB after: 4.7 MB



,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int16,True,False,True,False,80.0 B,1,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00
1,int_grob,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,10,0,9990,10000,0.0,44.62,40.0,90.0,446150.00
2,int_fein,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,255,0,9745,10000,-127.0,-0.21,0.0,127.0,-2140.00
3,float_summe,int16,pd.Int16,Int16,,True,False,True,True,30.0 B,1,823,0,9177,10000,-627.0,-201.43,-203.0,224.0,-2014255.00
4,first_name,str,pd.string,string,,False,True,True,True,628.0 B,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN
5,Letter1,str,pd.string,string,,False,True,True,True,580.0 B,1,24,0,9976,10000,A,NaN,NaN,Z,NaN
6,string_nonan,str,pd.string,string,,False,True,True,True,794.0 B,1,10000,0,0,10000,002z,NaN,NaN,üüÖpAI1,NaN
7,string_nan,str,pd.string,string,,False,True,True,True,629.0 B,1,8987,1013,0,10000,01ÄqpuZ,NaN,NaN,üüVmdp,NaN
8,int_nan,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,128,990,8882,10000,0.0,62.95,63.0,127.0,567156.00
9,float_nan,float32,np.float32,float32,,True,False,True,True,40.0 B,1,1000,2020,6980,10000,0.0,0.50,0.5,1.0,4011.86


## copy_datatype()

In [14]:
?pak.copy_datatype

Signature: pak.copy_datatype(data_to, data_from)
Docstring:
Copies the dtypes from data_from to data_to. 
Usable for Series and DataFrames.
When applied on a DataFrame, it's applied to all column names that match.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/pandas.py
Type:      function


In [15]:
# Target
df1 = df.copy()                      # Wrong datatypes
df1 = pak.drop_cols(df1,'int_fein')  # One column is missing

# Template
df2 = pak.change_datatype(df)        # Target datatypes
df2 = pak.drop_cols(df2,'City')      # One column is missing

change_datatype 
int_grob             --> pd.Int8   
int_fein             --> pd.Int8   
float_summe          --> pd.Int16  
int_nan              --> pd.Int8   
float_nan            --> np.float32
City                 --> pd.string 
change_datatype before: 4.9 MB after: 4.7 MB



In [16]:
result = pak.copy_datatype(df1, df2)

# Result: All columns except 'City' were treated
pak.analyse_cols(result)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int16,True,False,True,False,80.0 B,1,10000,0,0,10000,0.0,4999.50,4999.5,9999.0,49995000.00
1,int_grob,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,10,0,9990,10000,0.0,44.62,40.0,90.0,446150.00
2,float_summe,int16,pd.Int16,Int16,,True,False,True,True,30.0 B,1,823,0,9177,10000,-627.0,-201.43,-203.0,224.0,-2014255.00
3,first_name,str,pd.string,string,,False,True,True,True,619.0 B,1,435,0,9565,10000,Adolf,NaN,NaN,Zoe,NaN
4,Letter1,str,pd.string,string,,False,True,True,True,580.0 B,1,24,0,9976,10000,A,NaN,NaN,Z,NaN
5,string_nonan,str,pd.string,string,,False,True,True,True,696.0 B,1,10000,0,0,10000,002z,NaN,NaN,üüÖpAI1,NaN
6,string_nan,str,pd.string,string,,False,True,True,True,596.0 B,1,8987,1013,0,10000,01ÄqpuZ,NaN,NaN,üüVmdp,NaN
7,int_nan,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,128,990,8882,10000,0.0,62.95,63.0,127.0,567156.00
8,float_nan,float32,np.float32,float32,,True,False,True,True,40.0 B,1,1000,2020,6980,10000,0.0,0.50,0.5,1.0,4011.86
9,City,str,object,object,pd.string,False,False,True,True,490.0 B,1,2,3070,6928,10000,Bremen,NaN,NaN,Bremerhaven,NaN


## type_info()

In [17]:
?pak.type_info

Init signature: pak.type_info(search)
Docstring:     
Provides information about pandas types and standardises them.
Is initialised with anything, e.g. with the name of a class, or with the class itself.
Or, even better, with a series.
Ex:   i = type_info('Int32')     
      i.info()            # returns all attributes, including for example:
      i.class_object      # the class object
      i.name              # the name of the Dtype
      i.name_instance     # type of the contents of the series
      i.instance1         # an example instance that is not NaN              
File:           ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/type_info.py
Type:           type
Subclasses:     


In [18]:
# Call with class name
i = pak.type_info('uint32')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.uint32',
 'framework': 'np',
 'name_short': 'uint32',
 'name_long': 'np.uint32',
 'class_object': numpy.uint32,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': 0,
 'xmax': 4294967295}

In [19]:
# Call with class name
i = pak.type_info('intc')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.intc',
 'framework': 'np',
 'name_short': 'intc',
 'name_long': 'np.intc',
 'class_object': numpy.int32,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': -2147483648,
 'xmax': 2147483647}

In [20]:
# Call with class name
i = pak.type_info('category')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.category',
 'framework': 'pd',
 'name_short': 'category',
 'name_long': 'pd.Categorical',
 'class_object': pandas.core.arrays.categorical.Categorical,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [21]:
# Call with sloppy class name
i = pak.type_info('pd.String')     
i = pak.type_info('str')  
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.string',
 'framework': 'pd',
 'name_short': 'string',
 'name_long': 'pd.StringDtype',
 'class_object': pandas.core.arrays.string_.StringDtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [22]:
# Call with class name
i = pak.type_info('StringDtype')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.string',
 'framework': 'pd',
 'name_short': 'string',
 'name_long': 'pd.StringDtype',
 'class_object': pandas.core.arrays.string_.StringDtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': None,
 'xmax': None}

In [23]:
# Call with class name
i = pak.type_info('float16')     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.float16',
 'framework': 'np',
 'name_short': 'float16',
 'name_long': 'np.float16',
 'class_object': numpy.float16,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': -65500.0,
 'xmax': 65500.0}

In [24]:
# Call with class 
i = pak.type_info(np.int16)     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'np.int16',
 'framework': 'np',
 'name_short': 'int16',
 'name_long': 'np.int16',
 'class_object': numpy.int16,
 'is_hashable': True,
 'nan_allowed': False,
 'name_instance': '',
 'xmin': -32768,
 'xmax': 32767}

In [25]:
# Call with class
i = pak.type_info(pd.Int64Dtype)     
i.info()

{'instance1': None,
 'instance2': None,
 'name': 'pd.Int64',
 'framework': 'pd',
 'name_short': 'Int64',
 'name_long': 'pd.Int64Dtype',
 'class_object': pandas.core.arrays.integer.Int64Dtype,
 'is_hashable': True,
 'nan_allowed': True,
 'name_instance': '',
 'xmin': -9223372036854775808,
 'xmax': 9223372036854775807}

In [26]:
# Call with Series
pak.type_info(df.List).info()

{'instance1': ['Caroline',
  'Britta',
  'Werner',
  'Tom',
  'Arne',
  'Sven',
  'Monika',
  'Nils',
  'Tom',
  'Andreas'],
 'instance2': ['Helmut', 'Astrid', 'Richard', 'Tom', 'Niko', 'Leo'],
 'name': 'object',
 'framework': '',
 'name_short': 'object',
 'name_long': 'object',
 'class_object': object,
 'is_hashable': False,
 'nan_allowed': True,
 'name_instance': 'list',
 'xmin': None,
 'xmax': None}